In [1]:
import xarray as xr
import os
import pandas as pd
import numpy as np
import dask
import dask.array as da
import netCDF4
import zarr
import gcsfs
import esmpy
import xesmf as xe
import geopandas as gpd
import rioxarray
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import cartopy.crs as ccrs
from shapely.ops import transform
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pycountry_convert as pc
import glob
import calendar
import datetime
from shapely import wkt
from shapely.geometry import Point

In [2]:
path = "/Users/akawano/Library/CloudStorage/GoogleDrive-akawano@stanford.edu/My Drive/MyProjects/04_brick_kiln_emissions/emission_data"
dist_path = "/Users/akawano/Library/CloudStorage/GoogleDrive-akawano@stanford.edu/My Drive/MyProjects/04_brick_kiln_emissions"

In [3]:
# Import division (level 1) shapefiles
dist = gpd.read_file(os.path.join(dist_path, "bgd_adm_bbs_20201113_SHP/bgd_admbnda_adm1_bbs_20201113.shp"))
dist = dist[['ADM1_EN', 'geometry']].copy()
#dist = dist[(dist['ADM1_EN'] == "Khulna")|(dist['ADM1_EN'] == "Dhaka")] 
dist = dist.to_crs(7755)
dist.head()

,ADM1_EN,geometry
0,Barisal,"MULTIPOLYGON (((5062392.593 3797223.971, 50622..."
1,Chittagong,"MULTIPOLYGON (((5260268.454 3689637.669, 52603..."
2,Dhaka,"MULTIPOLYGON (((5050341.855 3930823.924, 50503..."
3,Khulna,"MULTIPOLYGON (((4938303.818 3779804.668, 49385..."
4,Mymensingh,"POLYGON ((4963048.479 4188982.858, 4963061.89 ..."


In [4]:
# Import kiln locations for those which participated in RCT
df1 = pd.read_csv(os.path.join(path, "gps_khulna_kilns.csv"))
df1 = df1[['kiln_id','latitude','longitude']].copy()
df1['category'] = "RCT"
print(df1.shape)
df1.head()                            

(590, 4)


,kiln_id,latitude,longitude,category
0,17,22.909920,89.149059,RCT
1,16,23.011150,89.215458,RCT
2,3,23.054592,88.926128,RCT
3,7,23.097718,89.292937,RCT
4,23,23.163530,89.361963,RCT


In [5]:
# Dhaka RCT kilns
df2 = pd.read_csv(os.path.join(path, "gps_dhaka_kilns.csv"))
df2 = df2[['kiln_id','latitude','longitude']].copy()
df2['category'] = "Scaling"
print(df2.shape)
#print(df2['kiln_id'].min())
df2.head()                            

(473, 4)


,kiln_id,latitude,longitude,category
0,26454,23.629603,90.441086,Scaling
1,26453,23.630494,90.427720,Scaling
2,26452,23.630976,90.429358,Scaling
3,26451,23.632267,90.426198,Scaling
4,26363,23.634706,90.427385,Scaling


In [6]:
print(df1.shape)
df = pd.concat([df1, df2])
print(df.shape)

(590, 4)
(1063, 4)


In [7]:
# create geopandas df from df
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]

# Create a GeoDataFrame using the geometry column
rct_kilns = gpd.GeoDataFrame(df, geometry=geometry, crs = 4326)
rct_kilns = rct_kilns.to_crs(7755).drop(columns = ['latitude','longitude'])
rct_kilns.head()

,kiln_id,category,geometry
0,17,RCT,POINT (4919851.782 3911492.283)
1,16,RCT,POINT (4925799.527 3922897.878)
2,3,RCT,POINT (4896473.825 3925735.083)
3,7,RCT,POINT (4932952.327 3932790.311)
4,23,RCT,POINT (4939397.309 3940380.672)


In [8]:
# Import all the kiln locations data
location = pd.read_csv(os.path.join(path, "model_kilns.csv")).drop(columns = ['label','prob','prediction'])

print(location.shape)
geometry = [Point(xy) for xy in zip(location['long'], location['lat'])]

# Create a GeoDataFrame using the geometry column
location_gdf = gpd.GeoDataFrame(location, geometry=geometry, crs = 4326)
location_gdf = location_gdf.to_crs(7755).drop(columns = ['lat','long'])
location_gdf['kiln_id'] = location_gdf.index + 100000
location_gdf['category'] = 'PNAS'
location_gdf.head()


(6978, 2)


,geometry,kiln_id,category
0,POINT (5241276.392 3739925.248),100000,PNAS
1,POINT (5241189.938 3740082.634),100001,PNAS
2,POINT (5240070.054 3740287.121),100002,PNAS
3,POINT (5241021.955 3740369.603),100003,PNAS
4,POINT (5239764.288 3751000.298),100004,PNAS


In [9]:
# PNAS kilns in two districts
location_filtered = gpd.sjoin(location_gdf, dist, how = 'left') 
location_filtered = location_filtered.drop(columns = 'index_right')
location_filtered

,geometry,kiln_id,category,ADM1_EN
0,POINT (5241276.392 3739925.248),100000,PNAS,Chittagong
1,POINT (5241189.938 3740082.634),100001,PNAS,Chittagong
2,POINT (5240070.054 3740287.121),100002,PNAS,Chittagong
3,POINT (5241021.955 3740369.603),100003,PNAS,Chittagong
4,POINT (5239764.288 3751000.298),100004,PNAS,Chittagong
...,...,...,...,...
6973,POINT (4839370.323 4275730.957),106973,PNAS,Rangpur
6974,POINT (4839913.778 4275959.103),106974,PNAS,Rangpur
6975,POINT (4882688.373 4281397.812),106975,PNAS,Rangpur
6976,POINT (4881679.449 4281437.967),106976,PNAS,Rangpur


In [10]:
# 1. Ensure both GeoDataFrames have the same CRS
rct_kilns = rct_kilns.to_crs(location_filtered.crs)

# check how many kilns have the exact same coordinates
# Convert geometries to a hashable representation using WKB (hex format)
rct_kilns_gdf = rct_kilns.copy()
location_gdf2 = location_filtered.copy()
rct_kilns_gdf['wkb_hex'] = rct_kilns_gdf['geometry'].apply(lambda geom: geom.wkb_hex)
location_gdf2['wkb_hex'] = location_gdf2['geometry'].apply(lambda geom: geom.wkb_hex)

# Merge the two GeoDataFrames on the WKB column to find exact matches
merged = rct_kilns_gdf.merge(location_gdf2, on='wkb_hex', suffixes=('_gdf1', '_gdf2'))

# Count how many identical geometries are found
print(f"Found {len(merged)} identical geometries")

# Optionally, if you want to see counts for each unique geometry:
common_counts = merged['wkb_hex'].value_counts()
print(common_counts)

Found 0 identical geometries
Series([], Name: count, dtype: int64)


In [11]:
# Create a buffer around each kiln location
# 100m
neighbor_distance = 100 
rct_kilns['buffer'] = rct_kilns.geometry.buffer(neighbor_distance)

# Set the 'buffer' column as the active geometry column
rct_kilns_buffer =rct_kilns.set_geometry('buffer')

# Now perform the spatial join using the active geometry from location_gdf_buffer.
neighbors_100m = gpd.sjoin(
    location_filtered, 
   rct_kilns_buffer, 
    how='inner', 
    predicate='within'
)

neighbors_100m = neighbors_100m[(neighbors_100m['ADM1_EN'] == 'Khulna')| (neighbors_100m['ADM1_EN'] == 'Dhaka')]
print(neighbors_100m.shape)
print(neighbors_100m['ADM1_EN'].unique())
neighbors_100m.head()


(775, 8)
['Khulna' 'Dhaka']


,geometry,kiln_id_left,category_left,ADM1_EN,index_right,kiln_id_right,category_right,geometry_right
330,POINT (4919351.571 3846693.509),100330,PNAS,Khulna,409,87046,RCT,POINT (4919336.766 3846713.934)
331,POINT (4919072.887 3846720.566),100331,PNAS,Khulna,508,87038,RCT,POINT (4919099.116 3846726.536)
343,POINT (4914846.657 3848691.974),100343,PNAS,Khulna,407,87044,RCT,POINT (4914835.204 3848678.278)
344,POINT (4915074.561 3848816.311),100344,PNAS,Khulna,507,87039,RCT,POINT (4915095.354 3848786.529)
384,POINT (4918750.111 3853611.274),100384,PNAS,Khulna,506,87042,RCT,POINT (4918746.334 3853602.185)


In [12]:
# Create a buffer around each kiln location
# 200m
neighbor_distance = 200 
rct_kilns['buffer'] = rct_kilns.geometry.buffer(neighbor_distance)

# Set the 'buffer' column as the active geometry column
rct_kilns_buffer =rct_kilns.set_geometry('buffer')

# Now perform the spatial join using the active geometry from location_gdf_buffer.
neighbors_200m = gpd.sjoin(
    location_filtered, 
   rct_kilns_buffer, 
    how='inner', 
    predicate='within'
)

neighbors_200m = neighbors_200m[(neighbors_200m['ADM1_EN'] == 'Khulna')| (neighbors_200m['ADM1_EN'] == 'Dhaka')]
print(neighbors_200m.shape)
print(neighbors_200m['ADM1_EN'].unique())
neighbors_200m.head()

(1228, 8)
['Khulna' 'Dhaka']


,geometry,kiln_id_left,category_left,ADM1_EN,index_right,kiln_id_right,category_right,geometry_right
330,POINT (4919351.571 3846693.509),100330,PNAS,Khulna,409,87046,RCT,POINT (4919336.766 3846713.934)
331,POINT (4919072.887 3846720.566),100331,PNAS,Khulna,508,87038,RCT,POINT (4919099.116 3846726.536)
334,POINT (4919223.706 3846865.575),100334,PNAS,Khulna,409,87046,RCT,POINT (4919336.766 3846713.934)
334,POINT (4919223.706 3846865.575),100334,PNAS,Khulna,508,87038,RCT,POINT (4919099.116 3846726.536)
343,POINT (4914846.657 3848691.974),100343,PNAS,Khulna,407,87044,RCT,POINT (4914835.204 3848678.278)


In [13]:
# Create a buffer around each kiln location

neighbor_distance = 300 
rct_kilns['buffer'] = rct_kilns.geometry.buffer(neighbor_distance)

# Set the 'buffer' column as the active geometry column
rct_kilns_buffer =rct_kilns.set_geometry('buffer')

# Now perform the spatial join using the active geometry from location_gdf_buffer.
neighbors_300m = gpd.sjoin(
    location_filtered, 
   rct_kilns_buffer, 
    how='inner', 
    predicate='within'
)

#print(neighbors_300m.shape)
neighbors_300m = neighbors_300m[(neighbors_300m['ADM1_EN'] == 'Khulna')| (neighbors_300m['ADM1_EN'] == 'Dhaka')]
print(neighbors_300m.shape)
print(neighbors_300m['ADM1_EN'].unique())
neighbors_300m.head()

(1760, 8)
['Khulna' 'Dhaka']


,geometry,kiln_id_left,category_left,ADM1_EN,index_right,kiln_id_right,category_right,geometry_right
330,POINT (4919351.571 3846693.509),100330,PNAS,Khulna,409,87046,RCT,POINT (4919336.766 3846713.934)
330,POINT (4919351.571 3846693.509),100330,PNAS,Khulna,508,87038,RCT,POINT (4919099.116 3846726.536)
331,POINT (4919072.887 3846720.566),100331,PNAS,Khulna,409,87046,RCT,POINT (4919336.766 3846713.934)
331,POINT (4919072.887 3846720.566),100331,PNAS,Khulna,508,87038,RCT,POINT (4919099.116 3846726.536)
334,POINT (4919223.706 3846865.575),100334,PNAS,Khulna,409,87046,RCT,POINT (4919336.766 3846713.934)


In [14]:
# unique kiln id
kilns_id = set(neighbors_200m['kiln_id_left'].tolist())
print(len(kilns_id))

# remove these from PNAS kilns
print(location_gdf.shape)
pnas_kilns = location_gdf[~location_gdf['kiln_id'].isin(kilns_id)]
pnas_kilns.shape

849
(6978, 3)


(6129, 3)

In [15]:
print(pnas_kilns.crs)
print(rct_kilns.crs)
kilns_gps = pd.concat([pnas_kilns, rct_kilns])
kilns_gps.shape

EPSG:7755
EPSG:7755


(7192, 4)

In [16]:
kilns_gps = kilns_gps.drop(columns = 'buffer')
kilns_gps

,geometry,kiln_id,category
0,POINT (5241276.392 3739925.248),100000,PNAS
1,POINT (5241189.938 3740082.634),100001,PNAS
2,POINT (5240070.054 3740287.121),100002,PNAS
3,POINT (5241021.955 3740369.603),100003,PNAS
4,POINT (5239764.288 3751000.298),100004,PNAS
...,...,...,...
468,POINT (5010246.922 4024388.481),26063,Scaling
469,POINT (5012036.789 4025280.612),26062,Scaling
470,POINT (5012152.109 4025286.948),26061,Scaling
471,POINT (5010412.9 4026648.095),26161,Scaling


In [17]:
kilns_gps = gpd.sjoin(kilns_gps, dist, how = 'left') 


In [18]:
kilns_gps = kilns_gps.drop(columns = 'index_right')
kilns_gps = kilns_gps.rename(columns = {'ADM1_EN':'division'})

kilns_gps

,geometry,kiln_id,category,division
0,POINT (5241276.392 3739925.248),100000,PNAS,Chittagong
1,POINT (5241189.938 3740082.634),100001,PNAS,Chittagong
2,POINT (5240070.054 3740287.121),100002,PNAS,Chittagong
3,POINT (5241021.955 3740369.603),100003,PNAS,Chittagong
4,POINT (5239764.288 3751000.298),100004,PNAS,Chittagong
...,...,...,...,...
468,POINT (5010246.922 4024388.481),26063,Scaling,Dhaka
469,POINT (5012036.789 4025280.612),26062,Scaling,Dhaka
470,POINT (5012152.109 4025286.948),26061,Scaling,Dhaka
471,POINT (5010412.9 4026648.095),26161,Scaling,Dhaka


In [19]:
kilns_gps.to_file(os.path.join(path, "gps_all_kilns.shp"))